# グラフェンの変形をできるだけおこさなくする。

(2024-11-05)

graphene-studio で生成した炭素配置をできるだけ維持するように、分子内の拘束条件(constraint)を減らすかわりに、原子を正しい位置に束縛(restraint)する。

## 注意

水分子モデルが TIP4P になっている。


## 1. 水和させる

graphene-studio で生成した構造の 1 つを選び、ファイル`graphene.gro`にコピーする。

その構造を水和させる。あらかじめ、セルの大きさは z 方向に 2 倍になっている。


In [ ]:
! cp diamond_95.itp graphene.itp
! cp diamond_95.gro graphene.gro
! gmx solvate -cs tip4p -cp graphene.gro -o solvated.gro -p solvated-fullfix.top 


## 2. 炭素原子をゆるく固定する。

(手順は`HMTA/`からもってきた。)

炭素を定位置に固定(restraint)する外場は`graphene-restraints.itp`ファイルに書く。こんな感じの拘束条件を原子の個数だけ書きならべる。

```shell
[ position_restraints ]
; 炭素を固定する。
;  i funct       fcx        fcy        fcz
   1    1       10000       10000       10000
   2    1       10000       10000       10000
...
```

原子の個数だけバネが必要なので長大なデータになる。自動生成させるのが良い。
以下のコードで、position_restraints を自動生成する。


In [ ]:
from graphenestudio.gromacs import read_gro

# 原子数を読みとるだけのためにgraphene.groを読みこんでいる。
with open("graphene.gro") as f:
    for frame in read_gro(f):
        Natom = frame.residue_id.shape[0]
        break

# 原子の個数だけバネをつける。
with open("graphene-restraints.itp", "w") as f:
    print(
        """[ position_restraints ]
; 炭素を固定する。
;  i funct       fcx        fcy        fcz""",
        file=f,
    )
    for i in range(Natom):
        print(f"{i+1} 1 10000 10000 10000", file=f)

あるいはコマンドラインで


In [ ]:
! python restrain.py < graphene.gro > graphene-restraints.itp

## 3. MD の準備

- `-r`オプションで固定先の座標を指定する。錨をおろす位置は継続計算でも変更してはいけない。
- シミュレーションの条件は`initial-solvated-fullfix.mdp`に書かれている。ここでは、圧力制御を`berendsen`とし、`semiisotropic`な変形を許す。
- `solvated-fullfix.top`の中で、上で作った`graphene-restraints.itp`が読みこまれることで、どの原子を固定するかが指示されている。
- 原子数があわないというエラーが出る場合は、`solvated-fullfix.top`ファイルに書きこまれた水和水分子数が正しいかどうかを確認する。
- 水分子モデルを変更する場合にも`solvated-fullfix.top`を編集する。


In [ ]:
! gmx grompp -maxwarn 2 -f initial-solvated-fullfix.mdp -p solvated-fullfix.top -o 00001.tpr -c solvated.gro -r solvated.gro

## 4. 初回の MD 実行


In [ ]:
! gmx mdrun -notunepme  -nt 8 -deffnm 00001 -c 00001-last.gro

In [ ]:
! make 00001-0.gro

## 5. 継続計算

継続する場合も、炭素原子固定位置を指示する部分はそのまま使う。

- `continue-solvated-fullfix.mdp`での変更点は
  1. 温度を 270 K に。
  2. extended_simulation に関する設定
     - `gen_vel                  = no`
     - `continuation             = yes`
  3. dt をまともな数字(0.001 ps)に。
  4. 圧力を 1 気圧に設定。

`Nose-Hoover`では異常に分子運動が抑えられてしまうので、やむをえず`v-rescale`を継続使用している。


`diff`コマンドで設定の違いを比較する。


In [ ]:
! diff initial-solvated-fullfix.mdp continue-solvated-fullfix.mdp

In [ ]:
! gmx grompp -maxwarn 2 -f continue-solvated-fullfix.mdp -p solvated-fullfix.top -o 00002.tpr -c 00001-last.gro -r solvated.gro

In [ ]:
! gmx mdrun -notunepme  -nt 8 -deffnm 00002 -c 00002-last.gro

In [ ]:
! make 00002-0.gro

## リングの可視化

analice2 を使う。


In [ ]:
! make 00002-0.yap

In [ ]:
# または直接入力
! analice2 00002-0.gro -O OW -H HW[12] -f rings[yaplot] > 00002-0.yap